In [8]:
import os
import yaml
import pandas as pd
import sqlalchemy
import time
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import display 
pd.options.display.max_columns = None
%matplotlib inline

def connect(poolclass=sqlalchemy.pool.QueuePool):
    with open(os.path.join(os.path.join('../..','config'), 'joco_db_profile.yaml')) as fd:
        config = yaml.load(fd)
        dburl = sqlalchemy.engine.url.URL(
            "postgres",
            host=config["host"],
            username=config["user"],
            database=config["db"],
            password=config["pass"],
            port=config["port"],
        )
        return sqlalchemy.create_engine(dburl, poolclass=poolclass)

    
conn = connect()

import SAVE_RecallAdjuster as sra
from importlib import reload
import datetime
from dateutil.relativedelta import relativedelta
import seaborn as sns

/home/hemank/.pyenv/versions/3.6.9/lib/python3.6/site-packages/ipykernel_launcher.py:15: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  from ipykernel import kernelapp as app


In [6]:
import datetime
from dateutil.relativedelta import relativedelta
base = datetime.datetime.strptime('2018-04-01', '%Y-%m-%d')
date_pairs = []
for x in range(9,-1,-1):
    date_pairs.append(
        (
        (base - relativedelta(months=4*x) - relativedelta(years=1)).strftime('%Y-%m-%d'),
        (base - relativedelta(months=4*x) - relativedelta(years=1)).strftime('%Y-%m-%d')
        )
    )
    date_pairs.append(
        (
        (base - relativedelta(months=4*x) - relativedelta(years=1)).strftime('%Y-%m-%d'),
        (base - relativedelta(months=4*x)).strftime('%Y-%m-%d')
        )
    )

import seaborn as sns
print(date_pairs)

[('2014-04-01', '2014-04-01'), ('2014-04-01', '2015-04-01'), ('2014-08-01', '2014-08-01'), ('2014-08-01', '2015-08-01'), ('2014-12-01', '2014-12-01'), ('2014-12-01', '2015-12-01'), ('2015-04-01', '2015-04-01'), ('2015-04-01', '2016-04-01'), ('2015-08-01', '2015-08-01'), ('2015-08-01', '2016-08-01'), ('2015-12-01', '2015-12-01'), ('2015-12-01', '2016-12-01'), ('2016-04-01', '2016-04-01'), ('2016-04-01', '2017-04-01'), ('2016-08-01', '2016-08-01'), ('2016-08-01', '2017-08-01'), ('2016-12-01', '2016-12-01'), ('2016-12-01', '2017-12-01'), ('2017-04-01', '2017-04-01'), ('2017-04-01', '2018-04-01')]


In [58]:
reload(sra)
start_time = time.time()
myRA_us_frac_3 = sra.RecallAdjuster(
        engine=conn,
        pg_role='hemank',
        schema='hemank_bias_zafar',
        experiment_hashes='None',
        date_pairs=date_pairs,
        list_sizes=[500],
        entity_demos='hemank_bias_alternatives.currmatch_entity_demos',
        demo_col='race_2way',
        start_model_id = 70,
        end_model_id = 71
)
print("Time Taken="+str(time.time()-start_time))

Not executing Entity Demos -- picking up from previously built schema.
Done creating entities
Setting demo values. Not using pre-set values
Not running subsampling OR bootstrap based models
Running Recall Adjustment
Time Taken=3.093769073486328


## Temp - Running Zafar ONLY to see their performance

In [20]:
import os, sys
import json
import datetime
import numpy as np
sys.path.insert(0, '../fair-classification/disparate_mistreatment/synthetic_data_demo')
sys.path.insert(0, '../fair-classification/fair_classification/')
from generate_synthetic_data import *
import utils as ut
import funcs_disp_mist as fdm
import plot_syn_boundaries as psb
import pandas as pd
import cvxpy
import yaml
from triage import create_engine
from triage.component.catwalk.estimators.classifiers import ScaledLogisticRegression
import time
from psycopg2.extras import Json

In [40]:
def label_top_k(distances, k):
    sort_df = pd.DataFrame({'dist': distances})
    sort_df.sort_values('dist', ascending=False, inplace=True)
    sort_df['pred_label'] = -1
    sort_df['orig_idx'] = sort_df.index
    sort_df.reset_index(inplace=True)
    i = k-1
    sort_df.loc[:i,'pred_label'] = 1
    sort_df.sort_values('orig_idx', inplace=True)
    
    return sort_df['pred_label'].values


def read_config_file(config_file):
    config = None
    try:
        with open (config_file, 'r') as file:
            config = yaml.safe_load(file)
    except Exception as e:
        print(e)
        print('Error reading the config file')
    return config

def connect(cred_folder):
    cred_file = os.path.join(cred_folder, 'joco_db_profile.yaml')
    db = read_config_file(cred_file)

    sql_engine = create_engine(
        'postgresql+psycopg2://%s:%s@%s:%i/%s'%(
            db['user'],
            db['pass'],
            db['host'],
            db['port'],
            db['db']
        )
    )
    return sql_engine

def load_matrix(FILE_PATH, matrix_id, entity_to_attrib, demo_col, label_col):
    df = pd.read_csv('%s/%s.csv.gz' % (FILE_PATH, matrix_id), compression='gzip')

    entity_col = []
    entities = df['entity_id'].values
    
    for i in range(len(entities)):
        try:
            attr = entity_to_attrib[int(entities[i])]
            entity_col.append(attr)
        except KeyError as e:
            entity_col.append("MISSING")

    df[demo_col] = entity_col
    df = df[df[demo_col]!='MISSING']
    df[label_col] = 2*(df[label_col] - 0.5)

    return df

def get_matrix_from_df(df, demo_col, label_col, cols_0):
    exclude_cols = ['entity_id', 'as_of_date', label_col, demo_col]
    
    for c in df.columns:
        if c in cols_0:
            exclude_cols.append(c)

    x = df[[c for c in df.columns if c not in exclude_cols]].values
    y = df[label_col].values
    x_control = {demo_col:df[demo_col].values}

    return x, y, x_control


def get_train_test_matrix_pairs(engine, experiment_hash, model_group_id):
    '''
    args:
        engine: PSQL Connection Engine
        experiment_hash: use the relevant experiment_hash attributing to the train/test matrices you want to obtain
        model_group_id: it might be better to use a model_group_id corresponding to 
        the dummy classifier for which train/test matrices might have been created; otherwise any relevant model_group_id
        will do.
    '''

    query = """
    with rel_models as
    (
        select model_id, model_hash 
        from model_metadata.models
        where 
            built_by_experiment = '%s'
            and model_group_id = %s
    ),
    train_matrices as
    (
        select model_id, matrix_uuid from
        train_results.prediction_metadata
    ),
    test_matrices as 
    (
        select model_id, matrix_uuid from 
        test_results.prediction_metadata
    ),
    matrix_info as 
    (
        select matrix_id, matrix_uuid,
        matrix_type, num_observations
        from model_metadata.matrices
    )
    select 
        rel_models.model_id, 
        train_matrices.matrix_uuid as train_matrix_id,
        test_matrices.matrix_uuid as test_matrix_id,
        m1.matrix_id as train_id, 
        m1.num_observations as train_n_obs,
        m2.matrix_id as test_id,
        m2.num_observations as test_n_obs
    from 
        rel_models, 
        train_matrices, test_matrices,
        matrix_info m1, matrix_info m2
    where
        rel_models.model_id = train_matrices.model_id 
    and
        rel_models.model_id = test_matrices.model_id
    and 
        m1.matrix_uuid = train_matrices.matrix_uuid
    and 
        m2.matrix_uuid = test_matrices.matrix_uuid;
    """%(str(experiment_hash), str(model_group_id))

    #print(query)
    df = pd.read_sql(query, engine)
    print(df.head(2))
    
    vals = df.values
    train_test_matrices = []
    
    for v in vals:
        train_test_matrices.append([(v[1], v[2])])

    return train_test_matrices

def get_entity_to_attrib_simple(conn, race_query, sa_column, anchor_sa_value):
    entity_to_attrib = {}
    df = pd.read_sql(race_query, conn)
    
    entity_ids = df['entity_id'].values
    sa_vals = df[sa_column].values
    
    attrib_info = {}

    for i in range(len(entity_ids)):
        if(sa_vals[i]==anchor_sa_value):
            sa_vals[i] = 1
        else:
            sa_vals[i] = 0.0
        attrib_info[int(entity_ids[i])] = sa_vals[i]
        
    print(pd.value_counts(sa_vals))
    return attrib_info

def calc_prec(pred_label, actual_label):
    label_pos = sum((pred_label == 1).astype(int))
    true_pos = sum(np.logical_and(pred_label == actual_label, pred_label == 1).astype(int))
    return float(true_pos/label_pos)


In [27]:
conn = connect('../../config')
experiment_hash='53918b13ea1115d6b5d2b5b16ef4e2f4'
race_query='select entity_id, race from hemank_bias_alternatives.currmatch_entity_demos'
label_col='booking_view_warr_bw_1y'
demo_col='race'
model_group_id=55
sa_column='race'
anchor_sa_value='W'
file_path='/mnt/data/experiment_data/peeps/joco_original/matrices'

In [30]:
train_test_matrices = get_train_test_matrix_pairs(conn, experiment_hash, model_group_id)
entity_to_attrib = get_entity_to_attrib_simple(conn, race_query, sa_column, anchor_sa_value)

   model_id                   train_matrix_id  \
0      1182  cf42d6c89aa0e48384c15088ebc7b289   
1      1233  df07e7bc5f1a151d38ff680f16410781   

                     test_matrix_id  \
0  44971539154916ae818762ba67595d33   
1  181d2f2f779c9215b1b78583f2aeae70   

                                            train_id  train_n_obs  \
0  booking_view_warr_bw_1y_binary_2013-04-01 00:0...        28658   
1  booking_view_warr_bw_1y_binary_2013-08-01 00:0...        28335   

                                             test_id  test_n_obs  
0  booking_view_warr_bw_1y_binary_2014-04-01 00:0...       27908  
1  booking_view_warr_bw_1y_binary_2014-08-01 00:0...       27534  
1.0    266050
0.0    171473
dtype: int64


In [ ]:
i=0
train_matrix_uuid = train_test_matrices[i][0][0]
test_matrix_uuid = train_test_matrices[i][0][1]

print(train_matrix_uuid)
print(test_matrix_uuid)

df_train = load_matrix(file_path, train_matrix_uuid, entity_to_attrib, demo_col, label_col)
df_train['as_of_date'] = pd.to_datetime(df_train['as_of_date'])
train_as_of_dates = df_train['as_of_date'].values
train_entity_ids = df_train['entity_id'].values



print(len(df_train.columns))

df_test = load_matrix(file_path, test_matrix_uuid, entity_to_attrib, demo_col, label_col)
df_test['as_of_date'] = pd.to_datetime(df_test['as_of_date'])
test_as_of_dates = df_test['as_of_date'].values
test_entity_ids = df_test['entity_id'].values

print(len(df_test.columns))

print("="*10)
print(train_entity_ids[0])
print(x_train[0])
print(test_entity_ids[0])
print(x_test[0])
print("="*10)
        
x_train, y_train, x_control_train = get_matrix_from_df(df_train, demo_col, label_col, [])
x_test, y_test, x_control_test = get_matrix_from_df(df_test, demo_col, label_col, [])

# Perform Scaled Logistic Regression to include only relevant features.
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
dsapp_lr = ScaledLogisticRegression()
dsapp_lr.fit(x_train, y_train)

print(dsapp_lr.coef_)
print(dsapp_lr.intercept_)

all_cols = [c for c in df_train.columns if c not in ['as_of_date', 'entity_id', demo_col, label_col]]
exclude_cols = []

for i, col in enumerate(all_cols):
    if dsapp_lr.coef_[0][i] == 0:
        exclude_cols.append(col)
#exclude_cols = keep_cols + ['intercept']

df_train['intercept'] = 1
df_test['intercept'] = 1

print(len(exclude_cols))
print(len(all_cols) - len(exclude_cols))

x_train, y_train, x_control_train = get_matrix_from_df(df_train, demo_col, label_col, exclude_cols)
x_test, y_test, x_control_test = get_matrix_from_df(df_test, demo_col, label_col, exclude_cols)

cf42d6c89aa0e48384c15088ebc7b289
44971539154916ae818762ba67595d33
3469
3469
100000080
[0. 0. 0. ... 0. 0. 1.]
100000080
[0. 0. 0. ... 0. 0. 1.]


In [53]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(28334, 2470) (27533, 2470) (28334,) (27533,)


In [55]:
x = x_train
y = y_train
x_control = x_control_train

max_iters = 150
max_iters_dccp = 75
        
num_points, num_features = x.shape
w = cvxpy.Variable(num_features)

np.random.seed(112233)
w.value = np.random.rand(x.shape[1])

constraints = []
loss = cvxpy.sum(  cvxpy.logistic( cvxpy.multiply(-y, x*w) )  ) / num_points
prob = cvxpy.Problem(cvxpy.Minimize(loss), constraints)

tau =  float(0.005)
mu = float(1.2)

loss_function = "logreg" # perform the experiments with logistic regression
EPS = float(1e-4)

        
prob.solve(method='dccp', tau=tau, mu=mu, tau_max=1e10, solver=cvxpy.ECOS, verbose=True, feastol=EPS, abstol=EPS, reltol=EPS, feastol_inacc=EPS, abstol_inacc=EPS, reltol_inacc=EPS,max_iters=max_iters, max_iter=max_iters_dccp)

ret_w = np.array(w.value).flatten()
sensitive_attrs = list(x_control_train.keys())

print("INPUT TRAIN:"+str(pd.value_counts(y_train)))
print("INPUT TEST:"+str(pd.value_counts(y_test)))


ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -3.679e+04  +2e+05  6e-01  2e+02  1e+00  1e+00    ---    ---    0  0  - |  -  - 
 1  +7.625e-01  -2.216e+04  +1e+05  2e-01  2e+02  6e-01  6e-01  0.5013  2e-01   2  1  2 |  4  3
 2  +1.018e+00  -1.749e+04  +1e+05  1e-01  1e+02  5e-01  5e-01  0.5013  6e-01   1  1  2 |  8  3
 3  +1.062e+00  -1.693e+04  +9e+04  1e-01  1e+02  5e-01  5e-01  0.9791  1e+00   1  1  1 | 20  0
 4  +1.256e+00  -1.353e+04  +8e+04  9e-02  1e+02  4e-01  4e-01  0.4010  5e-01   1  2  1 |  7  4
 5  +1.414e+00  -1.106e+04  +6e+04  7e-02  9e+01  3e-01  3e-01  0.9791  8e-01   1  2  2 | 12  0
 6  +1.475e+00  -1.024e+04  +6e+04  6e-02  8e+01  3e-01  3e-01  0.9791  9e-01   1  1  2 | 16  0
 7  +1.643e+00  -7.800e+03  +4e+04  4e-02  6e+01  2e-01  2e-01  0.4010  4e-01   2  2  2 |  6  4
 8  +1.836e+00  -5.485e+03  +3e+04  3e-02  4e+

28  +2.818e-01  +2.670e-01  +3e-01  6e-08  8e-03  6e-08  2e-06  0.7833  7e-01   2  1  1 |  9  1
29  +2.810e-01  +2.676e-01  +3e-01  6e-08  7e-03  6e-08  1e-06  0.9791  9e-01   2  1  1 | 15  0
30  +2.780e-01  +2.699e-01  +2e-01  3e-08  4e-03  3e-08  9e-07  0.5013  2e-01   2  1  1 |  4  3
31  +2.765e-01  +2.710e-01  +1e-01  2e-08  3e-03  2e-08  6e-07  0.7833  6e-01   2  1  1 |  8  1
32  +2.759e-01  +2.714e-01  +1e-01  2e-08  2e-03  2e-08  5e-07  0.9791  8e-01   2  1  1 | 12  0
33  +2.749e-01  +2.722e-01  +6e-02  1e-08  1e-03  1e-08  3e-07  0.5013  2e-01   1  1  1 |  4  3
34  +2.742e-01  +2.726e-01  +3e-02  7e-09  9e-04  7e-09  2e-07  0.9791  6e-01   2  0  0 |  8  0
35  +2.740e-01  +2.728e-01  +2e-02  5e-09  6e-04  5e-09  1e-07  0.9791  7e-01   1  1  1 | 10  0
36  +2.737e-01  +2.730e-01  +2e-02  3e-09  4e-04  3e-09  8e-08  0.5013  3e-01   1  1  1 |  5  3
37  +2.736e-01  +2.731e-01  +1e-02  2e-09  3e-04  2e-09  6e-08  0.9791  7e-01   2  0  0 | 10  0
38  +2.735e-01  +2.732e-01  +7e-03  1e-0

In [56]:
#train_score, test_score, cov_all_train, cov_all_test, s_attr_to_fp_fn_train, s_attr_to_fp_fn_test = fdm.get_clf_stats(ret_w, x_train, y_train, x_control, x_test, y_test, x_control_test, list(sensitive_attrs))
s_attr = sensitive_attrs[0]

print(s_attr)
print(pd.value_counts(x_control_test[s_attr]))
distances_boundary_test = fdm.get_distance_boundary(ret_w, x_test, x_control_test[s_attr])
print(pd.value_counts(y_test))
print(pd.value_counts(distances_boundary_test))

race
1.0    16913
0.0    10620
dtype: int64
-1.0    24155
 1.0     3378
dtype: int64
-0.020842      2
 1.574523      2
 2.255607      2
 2.031716      2
 0.746893      2
 0.852894      2
 1.477928      2
 1.457478      2
 1.585546      2
 1.891058      2
 2.245395      2
 1.113750      2
 1.566609      2
 2.458973      1
 0.667833      1
 0.860191      1
 2.274336      1
-18.014385     1
-0.462369      1
 3.184204      1
-2.566211      1
 0.826014      1
 3.181782      1
 1.639158      1
 1.401330      1
 2.820239      1
 0.956742      1
 2.396763      1
 1.697080      1
 1.333718      1
              ..
 2.902354      1
 2.780658      1
 1.408145      1
 1.055777      1
 1.098304      1
 54.307175     1
 1.472571      1
 5.431989      1
 1.429634      1
 2.009590      1
 2.148188      1
-14.046500     1
 1.043092      1
 4.610924      1
 0.607925      1
 120.917368    1
 1.089740      1
 1.289776      1
 1.713013      1
 1.237302      1
-0.051147      1
-9.415907      1
 1.296210     

In [57]:
k = 500
all_class_labels_assigned_test = label_top_k(distances_boundary_test, k)
prec_k = calc_prec(all_class_labels_assigned_test, y_test)
print('prec@%s_abs: %.5f' % (k, prec_k))

s_attr_to_fp_fn_test = fdm.get_fpr_fnr_sensitive_features(y_test, all_class_labels_assigned_test, x_control_test, sensitive_attrs, False)
        
    
for s_attr in s_attr_to_fp_fn_test.keys():
    for s_val in s_attr_to_fp_fn_test[s_attr].keys():
        s_attr_to_fp_fn_test[s_attr][s_val]['recall'] = 1.000-s_attr_to_fp_fn_test[s_attr][s_val]['fnr']

recall_nonwhite = s_attr_to_fp_fn_test['race'][0]['recall']
recall_white = s_attr_to_fp_fn_test['race'][1]['recall']


print('recall non-white: %.6f' % recall_nonwhite)
print('recall white: %.6f' % recall_white)
print('recall ratio: %.6f' % float(recall_white/recall_nonwhite))

prec@500_abs: 0.29000
doing:0.0
-1.0    9326
 1.0    1294
dtype: int64
------------------------------
-1    10524
 1       96
dtype: int64
------------------------------
ALL POS=1294
ALL NEG=9326
doing:1
-1.0    14829
 1.0     2084
dtype: int64
------------------------------
-1    16509
 1      404
dtype: int64
------------------------------
ALL POS=2084
ALL NEG=14829
recall non-white: 0.023957
recall white: 0.054702
recall ratio: 2.283388
